In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
import tensorflow as tf
from keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.layers import Dense, Conv2D, Flatten, Dropout,BatchNormalization,MaxPooling2D
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import Model
from keras.layers import Input, GlobalAveragePooling2D, BatchNormalization, Dropout, Dense
from tensorflow.keras.applications import EfficientNetB3
from keras.callbacks import ModelCheckpoint, EarlyStopping

**Creating Image Dict**

In [2]:
import csv

data = {}
with open('../input/state-farm-distracted-driver-detection/driver_imgs_list.csv') as f:
    reader = csv.reader(f)
    
    # first row in csv has column headers, we dont want column headers. 
    next(reader)
    
    # now getting our key-value pair
    for row in reader:
        key = row[1]
        if key in data:
            data[key].append(row[2])
        else:
            data[key] = [row[2]]

In [3]:
data.keys()

dict_keys(['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])

In [4]:
distraction_list = list(data.keys())

**Making Directories of Training and Testing**

In [5]:
# so we need to make master_data directory.
# inside it, we create two subdirectories training and testing.

import os
os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')


In [6]:
# inside the training and testing, we need to make 10 new subdirectories related to these distractions.
# so we iterate over each distractions.
for distraction in distraction_list:
    os.mkdir(os.path.join('master_data/training/', distraction))
    os.mkdir(os.path.join('master_data/testing/', distraction))
# we have created subdirectiories inside training and testing but the directories are empty so we need to fill them
# with source directory images.

**Inserting Particular Image on Particular Directory**

In [7]:
import shutil as sh
split_size = 0.8

for dis, images in data.items():
    train_size = int(split_size*len(images))
    train_images = images[:train_size]
    test_images = images[train_size:]
    for image in train_images:
        source = os.path.join('../input/state-farm-distracted-driver-detection/imgs/train',  dis,image)
        dest = os.path.join('master_data/training',   dis)
        sh.copy(source, dest)
    for image in test_images:
        source = os.path.join('../input/state-farm-distracted-driver-detection/imgs/train', dis, image)
        dest = os.path.join('master_data/testing',  dis)
        sh.copy(source, dest)

**Converting and Normalizing the Image using ImageDataGenerator**

In [8]:
train_dir = 'master_data/training'
test_dir = 'master_data/testing'

train_datagen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_datagen.flow_from_directory(
                                train_dir,   # directory from which we want to generate our data.
                                target_size = (224,224),  # telling what is the size of the class
                                class_mode =  'categorical',  # we need to tell what is the mode of classification, eg 'binary' 
                                batch_size = 64
)

test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_datagen.flow_from_directory(
                                test_dir,   # directory from which we want to generate our data.
                                target_size = (224,224),  # telling what is the size of the class
                                class_mode =  'categorical',  # we need to tell what is the mode of classification, eg 'binary' 
                                batch_size = 64
)

Found 17934 images belonging to 10 classes.
Found 4490 images belonging to 10 classes.


In [9]:
es = EarlyStopping(monitor = 'loss', patience = 2, min_delta=0.01)


**Defining the Model**

In [10]:
def define_model(num_classes):
    inputs = Input(shape=(224,224,3))
    base_model = EfficientNetB3(include_top=False, weights='imagenet')(inputs)
    x = GlobalAveragePooling2D()(base_model)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    output = Dense(units=num_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-4), 
                  loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model


model = define_model(10)
model.summary()

43941888/43941136 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb3 (Functional)  (None, None, None, 1536)  10783535  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1536)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1536)              6144      
_________________________________________________________________
dropout (Dropout)            (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                15370     
Total params: 10,805,049
Trainable params: 10,714,674
Non-trai

**Fitting the Model**

In [13]:
checkpoint_callback = ModelCheckpoint('best_model.hdf5', save_best_only=True, monitor='val_loss', mode='min')
es = EarlyStopping(monitor='val_loss', patience=5)
history = model.fit(train_generator, epochs=30,
                   callbacks=[es, checkpoint_callback])

Epoch 1/30
281/281 [==============================] - 216s 768ms/step - loss: 0.0063 - accuracy: 0.9987
Epoch 2/30
281/281 [==============================] - 217s 771ms/step - loss: 0.0048 - accuracy: 0.9991
Epoch 3/30
281/281 [==============================] - 216s 768ms/step - loss: 0.0026 - accuracy: 0.9995
Epoch 4/30
281/281 [==============================] - 216s 768ms/step - loss: 0.0026 - accuracy: 0.9994
Epoch 5/30
281/281 [==============================] - 217s 770ms/step - loss: 0.0020 - accuracy: 0.9996
Epoch 6/30
281/281 [==============================] - 215s 765ms/step - loss: 0.0020 - accuracy: 0.9996
Epoch 7/30
281/281 [==============================] - 216s 768ms/step - loss: 0.0082 - accuracy: 0.9978
Epoch 8/30
281/281 [==============================] - 216s 768ms/step - loss: 0.0072 - accuracy: 0.9978
Epoch 9/30
281/281 [==============================] - 216s 767ms/step - loss: 0.0049 - accuracy: 0.9987
Epoch 10/30
281/281 [==============================] - 217s 769m

**Model Evaluation**

In [14]:
model.evaluate(test_generator)

71/71 [==============================] - 27s 375ms/step - loss: 1.4548 - accuracy: 0.7111


[1.4548355340957642, 0.7111358642578125]